# Promptfoo Agent Red Teaming Steps File

## 1. Check for config file

The environment script creates a starter file at:
`$BESMAN_PROMPTFOO_CONFIG_PATH/promptfooconfig.yaml`

Open the starter file in your favourite editor.

Think of this file as the test plan. You tailor it; promptfoo executes it. Below is a comprehensive guide to every editable part.


### 1.1 Placeholder reference

| Placeholder | Meaning / What to Enter | Example |
|-------------|-------------------------|---------|
| `<AGENT_NAME>` | Descriptive name of the agent/system | `customer-support-assistant` |
| `<AGENT_LABEL>` | Short label for UI grouping | `support-api` |
| `<API_ENDPOINT_URL>` | Full HTTP endpoint to invoke | `https://api.example.com/v1/agent` |
| `<HTTP_METHOD>` | HTTP verb | `POST` |
| `<CONTENT_TYPE>` | Request content type | `application/json` |
| `<AUTHORIZATION_HEADER>` | Auth header (omit if none) | `Bearer ${API_TOKEN}` |
| `<REQUEST_BODY>` | JSON body string containing `{{prompt}}` token | `{"query":"{{prompt}}"}` |
| `<REQUEST_DELAY>` | Milliseconds between requests | `250` |
| `<TARGET_ID>` | Non-HTTP target id (file, provider, etc.) | `file://./agent.py:call_api` |
| `<DESCRIBE_YOUR_APPLICATION_PURPOSE>` | Narrative of agent purpose | Free-form text |
| `<YOUR_INDUSTRY_DOMAIN>` | Domain/sector | `Healthcare` |
| `<SYSTEM_RULES_AND_CONSTRAINTS>` | Operational & security boundaries | Bullet list |
| `<RED_TEAM_PERSONA>` | Attacker persona & goals | `Malicious insider seeking data exfil` |
| `<TEST_GENERATION_INSTRUCTIONS>` | Guidance for plugin test generation | Domain-specific heuristics |
| `<NUM_TESTS>` | Total tests parameter (see 2.9) | `25` |
| `<MAX_CONCURRENCY>` | Parallel requests (see 2.9) | `5` |

## 2. Deployment method

### 2.1. For locally running agents

#### Create a provider file

The `provider.py` file makes your agentic code compatible with promptfoo so promptfoo can invoke your agent.

Create `provider.py` in your agentic code base:
```python
from typing import Any, Dict
import json
# Optional: import your agent modules here

# Example synchronous wrapper. If your agent is async, you can run loop.run_until_complete.
def call_api(prompt: str, options: Dict[str, Any], context: Dict[str, Any]) -> Dict[str, Any]:
    try:
        # Access contextual info if needed (e.g., session id)
        session_id = context.get('vars', {}).get('sessionId')
        
        ########################

        # TODO: Invoke your agent here and store the result in result var:
        # result = #write your code here
        
        ##################################
        return {'output': result}
    except Exception as e:
        return {'error': f'call_api failure: {e}'}
```

Key requirements:
- Function name must be call_api `file://./provider.py:call_api`.
- Return a dict containing either `output` (success) or `error`.
- Avoid printing secrets—return minimal structured data.

#### Setting your target
```yaml
description: You can give your agent description
targets:
  - id: file:///<complete path to your provider>/provider.py
    label: Label for your agent
    config:
      verbose: true
```

#### Sample target config
```yaml
description: AI Travel Agent
targets:
  - id: file:///home/ubuntu/awesome-llm-apps/starter_ai_agents/ai_travel_agent/provider.py
    label: travel agent
    config:
      verbose: true
```

### 2.2. For API based

Here we are assuming your agent has some api exposed through which we can pass prompts

#### Setting your target

```yaml
description: Give your agent's description
targets:
  - id: http
    label: label for your agent
    config:
      url: <api url for sending prompts>
      method: POST
      headers:
        Content-Type: application/json
        Authorization: Bearer ${API_TOKEN}
      body: '{"query":"{{prompt}}"}'
    delay: 250
```

#### Sample target config for API based

```yaml
description: AI Newsletter Agent
targets:
  - id: http
    label: Newsletter Agent
    config:
      url: https://demo.dummy.com/call_api
      method: POST
      headers:
        Content-Type: application/json
      body: '{"message":"{{prompt}}"}'
```

## 3. Agent's Context

Fill the configuration file with details of your agent. Follow the comments given in the cofig file

- Application purpose
- Industry/Domain
- System rules/constraints
- Persona of Red team user

### 4. Plugins

Plugins are different categories of attack that you can perform on your agent. The config file lists a wide range of attacks, comment out or remove the ones that you dont need.



### 5. testGenerationInstructions

Give various instruction on how promptfoo should make prompts. You can give instructions like: Give realistic prompts with respect to the IT doman.

### 6. Strategies

Strategies are attack techniques that systematically probe LLM applications/Agents for vulnerabilities.

While plugins generate adversarial inputs, strategies determine how these inputs are delivered to maximize attack success rates.

Choose different strategies to really test your agents.

### 7. Number of test cases

`numTests:` give a number here for your ideal number of test case. 

### 8. Running red teaming

After you have done previous steps, save the config file. Move to the location where you saved the config file and run the below command.

`promptfoo redteam run`

### 9. Viewing result

To view the results in UI, run the below command,

`promptfoo view`

### 10. Downloading report and placing it in correct dir

1. Run `promptfoo view` to bring up the UI.
2. Click on `Results` in the ui.
3. Find the results of your current run.
4. Open the result.
5. Click on `View Vulnerability Report` button.
6. Click on the download icon and choose `json`
7. Move the file to this location `$PROMPTFOO_PATH`
8. From the location, rename the file using `mv <current file name> $DETAILED_REPORT_PATH`
9. Close the window